In [1]:
events = ['DoublePlay', 'Error', 'GroundOut', 'HBP/CatInt', 'HR', 'K',
          'LineOut/InfFly', 'Triple', 'Walk', 'LongSingle',
          'MediumSingle', 'ShortSingle', 'ShortDouble', 'LongDouble', 'LongFly',
          'MediumFly', 'ShortFly']

In [2]:
def next_state(current_state, event):
    outs, first, second, third = map(int, current_state)

    # DoublePlay
    if event == 'DoublePlay':
        if first and not second and third:
            outs += 2
            first, second, third = 0, 0, 0
        
        elif first and second and not third:
            outs += 2
            first, second, third = 0, 0, 1
        
        elif first and second and third:
            outs += 2
            first, second, third = 0, 1, 1
            
        elif first and not second and not third:
            outs += 2
            first, second, third = 0, 0, 0
        
        else:
            outs += 1

    # Error
    elif event == 'Error':
        third, second, first = second, first, 1

    # GroundOut
    elif event == 'GroundOut':
        if not first and not second and third:
            outs += 1
            first, second, third = 0, 0, 0
        
        elif not first and second and not third:
            outs += 1
            third = 1
            second = 0
            
        elif first and not second and third:
            outs += 1
            first, second, third = 1, 0, 0
        
        elif first and second and not third:
            outs += 1
            first, second, third = 1, 0, 1
        
        elif first and second and third:
            if outs == 0:
                outs += 1
                first, second, third = 1, 1, 1
            elif outs == 1:
                outs += 1
                first, second, third = 1, 0, 1

        else:
            outs += 1

    # HBP/CatInt
    elif event == 'HBP/CatInt':
        if first and not second and third:
            second = 1
            first = 1
        
        elif not first and not second and third:
            first = 1
            
        elif not first and second and not third:
            first = 1
            
        elif not first and second and third:
            first = 1
        
        else:
            third, second, first = second, first, 1

    # HR
    elif event == 'HR':
        first = second = third = 0

    # K
    elif event == 'K':
        outs += 1

    # LineOut/InfFly
    elif event == 'LineOut/InfFly':
        outs += 1

    # Triple
    elif event == 'Triple':
        first = second = third = 0
        third = 1

    # Walk
    elif event == 'Walk':
        if first and not second and third:
            second = 1
            first = 1
        
        elif not first and not second and third:
            first = 1
            
        elif not first and second and not third:
            first = 1
            
        elif not first and second and third:
            first = 1
        
        else:
            third, second, first = second, first, 1

    # Single(LongSingle, MediumSingle, ShortSingle)
    elif event == 'LongSingle':
        if not first and not second and not third:
            first, second, third = 1, 0, 0
        elif not first and second and not third:
            first, second, third = 1, 0, 0
        elif not first and not second and third:
            first, second, third = 1, 0, 0
        else:
            first, second, third = 1, 0, 1
    elif event == 'MediumSingle':
        if not first and not second and not third:
            first, second, third = 1, 0, 0
        elif not first and second and not third:
            first, second, third = 1, 0, 0
        elif not first and not second and third:
            first, second, third = 1, 0, 0
        elif not first and second and third:
            first, second, third = 1, 0, 0
        else:
            first, second, third = 1, 1, 0
    elif event == 'ShortSingle':
        third, second, first = third, second, 1

    # Double(ShortDouble, LongDouble)
    elif event == 'ShortDouble':
        if first and not second and not third:
            first, second, third = 1, 0, 1
        elif first and second and not third:
            first, second, third = 0, 1, 1
        elif first and not second and third:
            first, second, third = 0, 1, 1
        elif first and second and third:
            first, second, third = 0, 1, 1
        else:
            first, second, third = 0, 1, 0
    elif event == 'LongDouble':
        first, second, third = 0, 1, 0
    
    # FlyOut(LongFly, MediumFly, ShortFly)
    elif event == 'LongFly':
        if outs < 2:
            if (not first and not second and not third) or (not first and not second and third):
                first, second, third = 0, 0, 0
            elif (first and not second and not third) or (first and not second and third):
                first, second, third = 1, 0, 0
            elif (not first and second and not third) or (not first and second and third):
                first, second, third = 0, 0, 1
            elif (first and second and not third) or (first and second and third):
                first, second, third = 1, 0, 1
        outs += 1
    elif event == 'MediumFly':
        if third and outs < 2:
            third = 0
        outs += 1
    elif event == 'ShortFly':
        outs += 1

    # End Of Inning
    if outs >= 3:
        return '0000'

    # Set New State
    return f"{outs}{first}{second}{third}"

In [3]:
states = [f"{outs}{first}{second}{third}"
          for outs in range(3)
          for first in range(2)
          for second in range(2)
          for third in range(2)]

state_transition_matrix = {}

for state in states:
    state_transition_matrix[state] = {}
    for event in events:
        state_transition_matrix[state][event] = next_state(state, event)

In [4]:
import pandas as pd

df = pd.DataFrame.from_dict(state_transition_matrix, orient='index')
print(df)

     DoublePlay Error GroundOut HBP/CatInt    HR     K LineOut/InfFly Triple  \
0000       1000  0100      1000       0100  0000  1000           1000   0001   
0001       1001  0100      1000       0101  0000  1001           1001   0001   
0010       1010  0101      1001       0110  0000  1010           1010   0001   
0011       1011  0101      1011       0111  0000  1011           1011   0001   
0100       2000  0110      1100       0110  0000  1100           1100   0001   
0101       2000  0110      1100       0111  0000  1101           1101   0001   
0110       2001  0111      1101       0111  0000  1110           1110   0001   
0111       2011  0111      1111       0111  0000  1111           1111   0001   
1000       2000  1100      2000       1100  1000  2000           2000   1001   
1001       2001  1100      2000       1101  1000  2001           2001   1001   
1010       2010  1101      2001       1110  1000  2010           2010   1001   
1011       2011  1101      2011       11

In [5]:
str_df = df.astype(str)
print(str_df.dtypes)

DoublePlay        object
Error             object
GroundOut         object
HBP/CatInt        object
HR                object
K                 object
LineOut/InfFly    object
Triple            object
Walk              object
LongSingle        object
MediumSingle      object
ShortSingle       object
ShortDouble       object
LongDouble        object
LongFly           object
MediumFly         object
ShortFly          object
dtype: object


In [6]:
from pathlib import Path

BASE_DIR = Path.cwd()
output_path = BASE_DIR / "data" / "state_transition_matrix.csv"

str_df.to_csv(output_path, index_label='current_state')